<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/DeepLearning_Assignment_2_Shallow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%load_ext tensorboard

In [ ]:
! apt install ttf-mscorefonts-installer

In [ ]:
! fc-cache -f

In [ ]:
! fc-match Arial

### Loading MNIST

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(x_train_both, y_train_both), (x_test, y_test) = mnist.load_data()

print(x_train_both[x_train_both > 0])

x_train_both = x_train_both.reshape((x_train_both.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255

scaler = StandardScaler()
x_train_both = scaler.fit_transform(x_train_both)
x_test = scaler.transform(x_test)

print(x_train_both[x_train_both > 0])

x_train, x_validation, y_train, y_validation = train_test_split(x_train_both, y_train_both, test_size=0.2, shuffle=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_train_one_hot_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1)).toarray()
y_validation_hot_encoded = one_hot_encoder.transform(y_validation.reshape(-1, 1)).toarray()

In [ ]:
print(f"y_train: \n{y_train.reshape(-1, 1)[:5]}")
print(f"y_train_one_hot_encoded: \n{y_train_one_hot_encoded[:5]}")
print(f"y_validation_hot_encoded: \n{y_validation_hot_encoded[:3]}")

In [ ]:
print(x_train[0][:100])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### SGD Classifier

In [ ]:
! pip install tensorflow --upgrade

In [ ]:
! pip install keras-tuner --upgrade

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_train_one_hot_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1)).toarray()

### Keras Adam

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf

def plot_tensorflow_log(path):

    # Loading too much data is slow...
    tf_size_guidance = {
        # 'compressedHistograms': 100,
        # 'images': 0,
        # 'scalars': 10000,
        # 'histograms': 1000
        'tensors': 200
    }

    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()

    print(event_acc.Tags())

    training_accuracies = event_acc.Tensors('epoch_accuracy')
    tensor_np = tf.make_ndarray(training_accuracies[-1].tensor_proto)

    print(tensor_np)

    y = list()
    x = list()
    for i in range(len(training_accuracies)):
        y.append(tf.make_ndarray(training_accuracies[i].tensor_proto))
        x.append(i)

    return x, y

In [ ]:
experiment_train_size = 60000
X = x_train[:experiment_train_size]
y = y_train_one_hot_encoded[:experiment_train_size]

## Training

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers

def build_model(hp):
  num_units = hp.Choice("num_units", [1, 2, 4, 8, 16, 32])
  
  model = keras.Sequential()
  model.add(Dense(num_units, input_shape=(784,), activation='softmax'))
  model.add(Dense(10, input_shape=(784,), activation='softmax'))
  # model.add(Dense(10, input_shape=(784,), activation='softmax'))
  

  opt = keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
  return model

In [ ]:
import keras_tuner as kt

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=1000,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/deep-learning/shallow",
    project_name="num_units"
)

tuner.search(X, 
             y, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[tf.keras.callbacks.TensorBoard("/content/drive/MyDrive/deep-learning/shallow/num_units/callback")],
          
)

In [ ]:
! rm -rf /content/drive/MyDrive/deep-learning/shallow

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/deep-learning/shallow/num_units/callback

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt


log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0004/execution0/train/events.out.tfevents.1649282289.c90e60870e35.73.18.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='1 units - train')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0002/execution0/train/events.out.tfevents.1649282035.c90e60870e35.73.12.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='2 units - train')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0000/execution0/train/events.out.tfevents.1649281750.c90e60870e35.73.6.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='4 units - train')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0001/execution0/train/events.out.tfevents.1649281893.c90e60870e35.73.9.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='16 units - train')



log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0004/execution0/validation/events.out.tfevents.1649282294.c90e60870e35.73.19.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='1 units - validation')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0002/execution0/validation/events.out.tfevents.1649282040.c90e60870e35.73.13.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='2 units - validation')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0000/execution0/validation/events.out.tfevents.1649281755.c90e60870e35.73.7.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='4 units - validation')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0003/execution0/train/events.out.tfevents.1649282147.c90e60870e35.73.15.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, label='8 units - train')

log_file = "/content/drive/MyDrive/deep-learning/shallow/num_units/callback/0001/execution0/validation/events.out.tfevents.1649281898.c90e60870e35.73.10.v2"
x, y = plot_tensorflow_log(log_file)
plt.plot(x, y, '--', label='16 units - validation')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Training Progress - SGD")
plt.legend(loc='down right', frameon=True)
plt.show()